In [1]:
from __future__ import print_function
%matplotlib inline
%pylab inline
# %load_ext memory_profiler
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy
import itertools
from tabulate import tabulate
from pomegranate import *
from pomegranate import BayesianNetwork
import iac_org as iacO
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
attributes = ["gang", "age", "edu", "mar", "occ", "name"]

def prob(evidences, atts):
    count, total = 0, 0
    for datapoint in iacO.data:
        inc = True
        for key in evidences:
            inc = inc and datapoint[key] == evidences[key]
        if inc:
            total += 1
            f = True
            for key in atts:
                f = f and datapoint[key] == atts[key]
            if f:
                count += 1
    return count, total, float(count)/total

def enums(key):
    vals = set()
    for datapoint in iacO.data:
        if datapoint[key] not in vals:
            vals.add(datapoint[key])
    return list(vals)

In [51]:
import collections
noisy_p_feature_given_hidden = {}
c = 0
for name in enums(attributes[-1]):
    noisy_p_feature_given_hidden[name] = {}
    for att in attributes[:-1]:
#         pl = 0.1/(len(enums(att))-1)
        pl = 0
        for val in enums(att):
#             noisy_p_feature_given_hidden[name][val] = 0.9 if prob({"name":name},{att:val})[2] == 1.0 else pl
            noisy_p_feature_given_hidden[name][val] = abs(np.random.normal(0, 0.005, 1)[0])
            pl += noisy_p_feature_given_hidden[name][val] if prob({"name":name},{att:val})[2] == 1.0 else 0
#             c += 1 if prob({"name":name},{att:val})[2] == 1 else 0
        for val in enums(att):
            if prob({"name":name},{att:val})[2] == 1.0:
                noisy_p_feature_given_hidden[name][val] = 1-pl
            
print(prob({"name":"George"},{"edu":"JH"})[2], prob({"name":"George"},{"edu":"COL"})[2], prob({"name":"George"},{"edu":"HS"})[2])
print(c/27, noisy_p_feature_given_hidden["George"]["JH"], noisy_p_feature_given_hidden["George"]["COL"], noisy_p_feature_given_hidden["George"]["HS"])

1.0 0.0 0.0
0 0.9994731185413355 0.002615966322331199 0.0003124403469175942


In [52]:
distributions = {}
for att in attributes:
    probs = {}
    vals = enums(att)
    for val in vals:
        probs[val] = prob({},{att:val})[2]
    distributions[att] = DiscreteDistribution(probs)

dataInArray = []
n = {}
for datapoint in iacO.data:
    row = []
    for att in attributes:
        row.append(datapoint[att])
    dataInArray.append(row)
    
src = []
dist = []
states = []
for att in attributes:
    src.append(enums(att))
    dist.append(distributions[att])
print(len(dataInArray))
# print(tabulate(dataInArray))

27


In [53]:
# p(h_i|e) = p(h_i)*mul_j(p(e_j|h_i))/sum_k(p(h_k)mul_j(p(e_j|h_k)))

Noisy_CPT = []
d_evidence = {}
for element in itertools.product(*src[:-1]):
    evidences = element
    if evidences not in d_evidence:
        d_evidence[evidences] = {}
    for name in enums("name"):
        if name not in d_evidence[evidences]:
            d_evidence[evidences][name] = 1
        for evi in evidences:
            d_evidence[evidences][name] *= noisy_p_feature_given_hidden[name][evi]
    
eq_evidence = {}
for element in itertools.product(*src[:-1]):
    evidences = element
    eq_evidence[evidences] = 0
    d_evidence[evidences][name] = 27
    for name in enums("name"):
        eq_evidence[evidences] += d_evidence[evidences][name]

c = 0
Noisy_CPT_display = []
Noisy_CPT_dict = {}
for element in itertools.product(*src):
    name = element[-1]
    evidences = element[:-1]
    temp = list(element)
    normalized_noisy_prob = d_evidence[evidences][name]/eq_evidence[evidences]
    Noisy_CPT.append(temp+[normalized_noisy_prob])
    if temp in dataInArray:
        Noisy_CPT_display.append(temp+[normalized_noisy_prob]+[1])
        Noisy_CPT_dict[element] = (normalized_noisy_prob, 1)
        c += 1
    else:
        Noisy_CPT_display.append(temp+[normalized_noisy_prob]+[0])
        Noisy_CPT_dict[element] = (normalized_noisy_prob, 0)

print(tabulate(Noisy_CPT_display, headers=attributes+["noisy prob", "actual"]))

gang    age    edu    mar       occ      name      noisy prob    actual
------  -----  -----  --------  -------  ------  ------------  --------
Jets    20s    HS     single    bookie   Pete     0.0352451           1
Jets    20s    HS     single    bookie   Ned      6.63361e-13         0
Jets    20s    HS     single    bookie   Jim      1.46285e-09         0
Jets    20s    HS     single    bookie   Ralph    2.79112e-09         0
Jets    20s    HS     single    bookie   Nick     2.70651e-10         0
Jets    20s    HS     single    bookie   Clyde    3.83589e-07         0
Jets    20s    HS     single    bookie   Rick     1.44985e-12         0
Jets    20s    HS     single    bookie   John     2.07542e-10         0
Jets    20s    HS     single    bookie   Lance    1.76277e-10         0
Jets    20s    HS     single    bookie   Ken      2.64604e-07         0
Jets    20s    HS     single    bookie   Mike     2.34011e-07         0
Jets    20s    HS     single    bookie   Fred     9.54202e-05   

In [55]:
married_status = enums("mar")
names = enums("name")
base_case = ["Jets", "20s", "HS", "single", "bookie", "Pete"]
mSum = 0
nSum = 0
calSum = 0
for stat in married_status:
    new_case = base_case[:3]+[stat]+base_case[4:]
    name = new_case[-1]
    cal = 1
    vals = []
    for val in new_case[:-1]:
        vals.append(noisy_p_feature_given_hidden[name][val])
        cal *= noisy_p_feature_given_hidden[name][val]
    calSum += cal
    print(new_case, Noisy_CPT_dict[tuple(new_case)][1], Noisy_CPT_dict[tuple(new_case)][0], cal, vals)
    mSum += Noisy_CPT_dict[tuple(new_case)][0]

print(mSum, calSum, 0.9*0.9*0.9*0.9)

eSum = 0
for element in itertools.product(*src[:-1]):
    temp = list(element)
    new_case = temp+["Pete"]
    print(new_case, Noisy_CPT_dict[tuple(new_case)][0], Noisy_CPT_dict[tuple(new_case)][1])
    eSum += Noisy_CPT_dict[tuple(new_case)][0]

print(eSum)
    
for name in names:
    new_case = base_case[:-1]+[name]
    print(new_case, Noisy_CPT_dict[tuple(new_case)][1], Noisy_CPT_dict[tuple(new_case)][0])
    nSum += Noisy_CPT_dict[tuple(new_case)][0]

print(nSum)

['Jets', '20s', 'HS', 'single', 'bookie', 'Pete'] 1 0.03524509484791581 0.9870530607372586 [0.9984217209896039, 0.9961764557151023, 0.9955586223932762, 0.9968498303274843, 0.9999853289747856]
['Jets', '20s', 'HS', 'married', 'bookie', 'Pete'] 0 7.291117646759182e-05 0.0019688029612935946 [0.9984217209896039, 0.9961764557151023, 0.9955586223932762, 0.0019883438651695636, 0.9999853289747856]
['Jets', '20s', 'HS', 'divorced', 'bookie', 'Pete'] 0 0.00017355326890769763 0.004686763436882861 [0.9984217209896039, 0.9961764557151023, 0.9955586223932762, 0.0047332808363433725, 0.9999853289747856]
0.0354915592932911 0.9937086271354352 0.6561
['Jets', '20s', 'HS', 'single', 'bookie', 'Pete'] 0.03524509484791581 1
['Jets', '20s', 'HS', 'single', 'burglar', 'Pete'] 2.7879231975200684e-05 0
['Jets', '20s', 'HS', 'single', 'pusher', 'Pete'] 0.0002626996028329124 0
['Jets', '20s', 'HS', 'married', 'bookie', 'Pete'] 7.291117646759182e-05 0
['Jets', '20s', 'HS', 'married', 'burglar', 'Pete'] 5.559678196

In [56]:
cond = ConditionalProbabilityTable(Noisy_CPT, dist[:-1])
name = State(cond, name="name")
gang = State(distributions["gang"], name="gang")
age = State(distributions["age"], name="age")
edu = State(distributions["edu"], name="edu")
mar = State(distributions["mar"], name="mar")
occ = State(distributions["occ"], name="occ")

model = BayesianNetwork("Jets and Sharks")
model.add_states(gang, age, edu, mar, occ, name)

model.add_edge(gang, name)
model.add_edge(age, name)
model.add_edge(edu, name)
model.add_edge(mar, name)
model.add_edge(occ, name)
model.bake()

In [58]:
# print(model.predict_proba({'gang': 'Sharks', 'age': '20s'})) #Content Addressability
# print(model.predict_proba({'gang': 'Sharks', 'age': '30s', 'edu': 'JH', 'occ': 'bookie', 'mar': 'divorced'})) #Graceful Degradation. is actually 'mar': 'single'
# print(model.predict_proba({'name': 'Ike'})) #Retrieval by name
print(model.predict_proba({'gang': 'Jets'})) #Spontaneous Generalization
# print(model.predict_proba({'mar': 'single'})) #Spontaneous Generalization

['Jets'
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "20s" :0.3703703703703702,
            "30s" :0.4814814814814816,
            "40s" :0.1481481481481484
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "HS" :0.40740740740740716,
            "JH" :0.3703703703703706,
            "COL" :0.22222222222222213
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "single" :0.4814814814814807,
            "married" :0.37037037037037085,
            "divorced" :0.14814814814814836
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "pusher" :0.3333333333333334,
            "b

In [42]:
# print(prob({'gang': 'Jets', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'occ': 'bookie' ,'mar': 'single'}))
# print(prob({'name': 'Lance', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'name': 'Lance', 'occ': 'pusher'}))
# print(model.predict_proba({'gang': 'Sharks', 'age': '30s', 'edu': 'JH', 'occ': 'bookie', 'mar': 'married'})) #'mar': 'single'
# print(model.predict_proba({'gang': 'Jets'}))
# print(model.predict_proba({'name': 'John'}))
# print(prob({},{'gang': 'Jets'}))
# print(prob({},{'name': 'John'x}))

[{
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "Jets" :0.08623311650635523,
            "Sharks" :0.9137668834936448
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "20s" :0.19633324654467124,
            "30s" :0.7470914357720547,
            "40s" :0.056575317683274055
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "HS" :0.03456385399380643,
            "JH" :0.888417057001044,
            "COL" :0.07701908900514963
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "single" :0.7724708333239728,
            "married" :0.13035334984189637,
            "divorced" :